<a href="https://colab.research.google.com/github/raakanaka/Analytica-Sentiment-Data/blob/main/Scraping_Post_Native_PHP_to_Wordpress.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
from requests.auth import HTTPBasicAuth
import datetime

# --- SETUP WP ---
WP_URL = 'https://https://aidig-system.com/dinas-kehutanan/wp-json/wp/v2/posts'
WP_USER = 'client'
WP_APP_PASSWORD = 'client123'

# --- SCRAPE BERITA ---
BERITA_URL = 'https://dishut.kalteng.go.id/berita'
resp = requests.get(BERITA_URL)
soup = BeautifulSoup(resp.content, 'html.parser')

# Sesuaikan dengan struktur aktual
artikel_divs = soup.find_all('div', class_='elementor-post')  # atau apapun class-nya

for div in artikel_divs:
    # Judul dan URL
    link_tag = div.find('a', href=True)
    title = link_tag.get_text(strip=True)
    article_url = link_tag['href']

    # Kunjungi halaman artikel
    artikel_resp = requests.get(article_url)
    artikel_soup = BeautifulSoup(artikel_resp.content, 'html.parser')

    # Ambil isi artikel (sesuaikan selector-nya)
    content_div = artikel_soup.find('div', class_='elementor-widget-theme-post-content')
    content_html = str(content_div) if content_div else '<p>[Konten tidak ditemukan]</p>'

    # Ambil tanggal jika ada, atau pakai hari ini
    tanggal = datetime.datetime.now().isoformat()

    # --- POST ke WordPress ---
    post_data = {
        'title': title,
        'content': content_html,
        'status': 'publish',
        'date': tanggal,
    }

    res = requests.post(WP_URL, json=post_data, auth=HTTPBasicAuth(WP_USER, WP_APP_PASSWORD))

    if res.status_code == 201:
        print(f"✅ Berhasil posting: {title}")
    else:
        print(f"❌ Gagal posting: {title} ({res.status_code})")
        print(res.text)
